# 식습관 변화 기반 바이오마커 변화 예측 모델

## 개요
- **목적**: 식습관 변화가 바이오마커 변화에 미치는 영향을 직접 모델링
- **방법**: 종단 데이터의 시점 간 변화량(Δ) 분석
- **특징**: 인구통계 변수의 영향을 제어하여 순수한 식습관 효과 추출
- **차이점**: 기존 모델은 절대값 예측, 새 모델은 **변화량** 예측

## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import shap
shap.initjs()

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8')

from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

print("🚀 식습관 변화 예측 모델 초기화 완료")

In [ ]:
# 데이터 로드
df = pd.read_excel('../data/total_again.xlsx', index_col='R-ID').drop(columns=['Unnamed: 0'])

print(f"\n=== 원본 데이터 개요 ===")
print(f"총 데이터 수: {len(df):,}건")
print(f"참여자 수: {len(df.index.unique()):,}명")
print(f"변수 수: {len(df.columns)}개")

# 날짜 변환
df['수진일'] = pd.to_datetime(df['수진일'])
df = df.sort_values(['R-ID', '수진일'])

# 참여자당 방문 횟수
visit_counts = df.groupby(df.index).size()
print(f"평균 방문 횟수: {visit_counts.mean():.2f}회")
print(f"방문 횟수 분포:")
print(visit_counts.value_counts().sort_index())

# 2회 이상 방문자만 선택 (변화 분석 가능)
multi_visit_patients = visit_counts[visit_counts >= 2].index
print(f"\n✅ 2회 이상 방문자: {len(multi_visit_patients):,}명")

df.head(3)

## 2. 변화량 데이터 생성

### 전략:
1. 동일 환자의 연속 방문 간 변화 계산
2. Δ(바이오마커) = 바이오마커(t2) - 바이오마커(t1)
3. Δ(식습관) = 식습관(t2) - 식습관(t1)
4. 시간 간격도 고려 (months_between)

In [ ]:
# 식습관 변수
diet_vars = ['간식빈도', '고지방 육류', '곡류', '과일', '단맛', '단백질류', '물', '밥 양', 
             '식사 빈도', '식사량', '외식빈도', '유제품', '음료류', '인스턴트 가공식품', 
             '짠 간', '짠 식습관', '채소', '커피', '튀김']

# 바이오마커 변수
biomarker_vars = ['체중', '체질량지수', '허리둘레(WAIST)', 'SBP', 'DBP', 
                  'GLUCOSE', 'HBA1C', 'TG', 'HDL CHOL.', 'LDL CHOL.', 'eGFR']

# 인구통계 변수 (고정 또는 천천히 변하는 변수)
demo_vars = ['나이', '성별', '신장']

# 생활습관 변수
lifestyle_vars = ['일반담배_흡연여부', '활동량', '음주']

print(f"📋 변수 정의:")
print(f"   식습관: {len(diet_vars)}개")
print(f"   바이오마커: {len(biomarker_vars)}개")
print(f"   인구통계: {len(demo_vars)}개")
print(f"   생활습관: {len(lifestyle_vars)}개")

In [ ]:
def create_change_dataset(df, diet_vars, biomarker_vars, demo_vars, lifestyle_vars, 
                         min_months=3, max_months=24):
    """
    연속 방문 간 변화량 데이터셋 생성
    
    Parameters:
    - min_months: 최소 시간 간격 (너무 짧으면 변화 미미)
    - max_months: 최대 시간 간격 (너무 길면 다른 요인 개입)
    """
    
    change_records = []
    
    for patient_id in df.index.unique():
        patient_data = df.loc[df.index == patient_id].sort_values('수진일')
        
        if len(patient_data) < 2:
            continue
        
        # 연속 방문 쌍 생성
        for i in range(len(patient_data) - 1):
            visit1 = patient_data.iloc[i]
            visit2 = patient_data.iloc[i + 1]
            
            # 시간 간격 계산 (개월)
            days_between = (visit2['수진일'] - visit1['수진일']).days
            months_between = days_between / 30.44
            
            # 시간 간격 필터
            if months_between < min_months or months_between > max_months:
                continue
            
            record = {
                'patient_id': patient_id,
                'months_between': months_between,
                'days_between': days_between
            }
            
            # 인구통계 변수 (baseline)
            for var in demo_vars:
                if var in visit1.index:
                    record[f'{var}_baseline'] = visit1[var]
            
            # 생활습관 변화
            for var in lifestyle_vars:
                if var in visit1.index and var in visit2.index:
                    val1 = pd.to_numeric(visit1[var], errors='coerce')
                    val2 = pd.to_numeric(visit2[var], errors='coerce')
                    if pd.notna(val1) and pd.notna(val2):
                        record[f'Δ{var}'] = val2 - val1
            
            # 식습관 변화 (Δdiet)
            diet_changes = []
            for var in diet_vars:
                if var in visit1.index and var in visit2.index:
                    val1 = pd.to_numeric(visit1[var], errors='coerce')
                    val2 = pd.to_numeric(visit2[var], errors='coerce')
                    
                    if pd.notna(val1) and pd.notna(val2):
                        change = val2 - val1
                        record[f'Δ{var}'] = change
                        record[f'{var}_t1'] = val1  # baseline 식습관도 포함
                        diet_changes.append(abs(change))
            
            # 식습관 변화의 크기 요약
            if diet_changes:
                record['diet_change_magnitude'] = np.mean(diet_changes)
                record['diet_change_max'] = np.max(diet_changes)
            
            # 바이오마커 변화 (target)
            for var in biomarker_vars:
                if var in visit1.index and var in visit2.index:
                    val1 = pd.to_numeric(visit1[var], errors='coerce')
                    val2 = pd.to_numeric(visit2[var], errors='coerce')
                    
                    if pd.notna(val1) and pd.notna(val2):
                        record[f'Δ{var}'] = val2 - val1
                        record[f'{var}_t1'] = val1  # baseline 바이오마커
            
            change_records.append(record)
    
    change_df = pd.DataFrame(change_records)
    
    return change_df

# 변화량 데이터 생성
print("🔄 변화량 데이터 생성 중...")
change_df = create_change_dataset(
    df, diet_vars, biomarker_vars, demo_vars, lifestyle_vars,
    min_months=3, max_months=24
)

print(f"\n✅ 변화량 데이터 생성 완료")
print(f"   총 레코드 수: {len(change_df):,}개")
print(f"   고유 환자 수: {change_df['patient_id'].nunique():,}명")
print(f"   평균 시간 간격: {change_df['months_between'].mean():.1f}개월")
print(f"   시간 간격 범위: {change_df['months_between'].min():.1f}~{change_df['months_between'].max():.1f}개월")

change_df.head()

## 3. 탐색적 데이터 분석 (EDA)

In [ ]:
# 식습관 변화 분포
diet_change_cols = [col for col in change_df.columns if col.startswith('Δ') and 
                    any(diet in col for diet in diet_vars)]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 식습관 변화 크기 분포
axes[0, 0].hist(change_df['diet_change_magnitude'].dropna(), bins=50, edgecolor='black')
axes[0, 0].set_xlabel('평균 식습관 변화 크기')
axes[0, 0].set_ylabel('빈도')
axes[0, 0].set_title('식습관 변화 크기 분포')
axes[0, 0].axvline(change_df['diet_change_magnitude'].median(), 
                   color='red', linestyle='--', label=f'중앙값: {change_df["diet_change_magnitude"].median():.2f}')
axes[0, 0].legend()

# 2. 시간 간격 vs 식습관 변화
axes[0, 1].scatter(change_df['months_between'], change_df['diet_change_magnitude'], 
                   alpha=0.3, s=10)
axes[0, 1].set_xlabel('시간 간격 (개월)')
axes[0, 1].set_ylabel('평균 식습관 변화 크기')
axes[0, 1].set_title('시간 간격 vs 식습관 변화')

# 3. SBP 변화 분포
if 'ΔSBP' in change_df.columns:
    sbp_changes = change_df['ΔSBP'].dropna()
    axes[1, 0].hist(sbp_changes, bins=50, edgecolor='black', color='salmon')
    axes[1, 0].set_xlabel('SBP 변화 (mmHg)')
    axes[1, 0].set_ylabel('빈도')
    axes[1, 0].set_title(f'SBP 변화 분포 (평균: {sbp_changes.mean():.1f}mmHg)')
    axes[1, 0].axvline(0, color='red', linestyle='--', label='변화 없음')
    axes[1, 0].legend()

# 4. 주요 바이오마커 변화 요약
biomarker_change_cols = ['ΔSBP', 'ΔDBP', 'ΔGLUCOSE', 'ΔHBA1C']
available_biomarkers = [col for col in biomarker_change_cols if col in change_df.columns]

if available_biomarkers:
    means = [change_df[col].mean() for col in available_biomarkers]
    axes[1, 1].bar(range(len(available_biomarkers)), means, color='skyblue', edgecolor='black')
    axes[1, 1].set_xticks(range(len(available_biomarkers)))
    axes[1, 1].set_xticklabels([col.replace('Δ', '') for col in available_biomarkers], rotation=45)
    axes[1, 1].set_ylabel('평균 변화량')
    axes[1, 1].set_title('주요 바이오마커 평균 변화')
    axes[1, 1].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n📊 변화량 통계:")
print(f"   식습관 변화 있는 레코드: {(change_df['diet_change_magnitude'] > 0.5).sum():,}개 "
      f"({(change_df['diet_change_magnitude'] > 0.5).sum() / len(change_df) * 100:.1f}%)")
print(f"   SBP 증가: {(change_df.get('ΔSBP', pd.Series([0])) > 5).sum():,}개")
print(f"   SBP 감소: {(change_df.get('ΔSBP', pd.Series([0])) < -5).sum():,}개")

## 4. 변화량 예측 모델 훈련

### 모델 설계:
- **입력**: Δ식습관, baseline 바이오마커, baseline 식습관, 인구통계, 시간 간격
- **출력**: Δ바이오마커
- **특징**: 인구통계는 고정값으로 사용 (변화 예측에는 영향 제한적)

In [ ]:
def train_change_model(change_df, target_biomarker, diet_vars):
    """
    특정 바이오마커 변화 예측 모델 훈련
    """
    
    target_col = f'Δ{target_biomarker}'
    baseline_col = f'{target_biomarker}_t1'
    
    if target_col not in change_df.columns:
        print(f"❌ {target_biomarker}: 데이터 없음")
        return None
    
    # 특성 선택
    feature_cols = []
    
    # 1. 식습관 변화 (핵심 특성)
    for diet_var in diet_vars:
        delta_col = f'Δ{diet_var}'
        baseline_diet_col = f'{diet_var}_t1'
        if delta_col in change_df.columns:
            feature_cols.append(delta_col)
        if baseline_diet_col in change_df.columns:
            feature_cols.append(baseline_diet_col)
    
    # 2. baseline 바이오마커
    if baseline_col in change_df.columns:
        feature_cols.append(baseline_col)
    
    # 3. 인구통계 (고정값)
    demo_cols = ['나이_baseline', '성별_baseline']
    for col in demo_cols:
        if col in change_df.columns:
            feature_cols.append(col)
    
    # 4. 시간 간격
    feature_cols.append('months_between')
    
    # 5. 생활습관 변화
    lifestyle_change_cols = [col for col in change_df.columns 
                            if col.startswith('Δ') and any(life in col for life in ['담배', '활동량', '음주'])]
    feature_cols.extend(lifestyle_change_cols)
    
    # 데이터 준비
    X = change_df[feature_cols].copy()
    y = change_df[target_col].copy()
    
    # 결측치 및 이상치 처리
    valid_mask = y.notna() & (y.abs() < y.abs().quantile(0.98))
    X = X[valid_mask]
    y = y[valid_mask]
    
    # 범주형 변수 처리
    if '성별_baseline' in X.columns:
        X['성별_baseline'] = X['성별_baseline'].map({'M': 1, 'F': 0}).fillna(0)
    
    # 수치형 변환
    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)
    
    if len(X) < 100:
        print(f"❌ {target_biomarker}: 데이터 부족 ({len(X)}개)")
        return None
    
    print(f"\n🎯 {target_biomarker} 모델 훈련")
    print(f"   데이터: {len(X):,}개")
    print(f"   특성: {len(feature_cols)}개")
    
    # Train/Test 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # 표준화
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 모델 훈련
    models = {}
    
    # Random Forest
    rf_model = RandomForestRegressor(
        n_estimators=200, max_depth=10, min_samples_split=10,
        min_samples_leaf=5, random_state=42, n_jobs=-1
    )
    rf_model.fit(X_train_scaled, y_train)
    rf_pred = rf_model.predict(X_test_scaled)
    models['rf'] = {
        'model': rf_model,
        'r2': r2_score(y_test, rf_pred),
        'mae': mean_absolute_error(y_test, rf_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, rf_pred))
    }
    
    # XGBoost
    xgb_model = xgb.XGBRegressor(
        n_estimators=150, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, n_jobs=-1, verbosity=0
    )
    xgb_model.fit(X_train_scaled, y_train)
    xgb_pred = xgb_model.predict(X_test_scaled)
    models['xgb'] = {
        'model': xgb_model,
        'r2': r2_score(y_test, xgb_pred),
        'mae': mean_absolute_error(y_test, xgb_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, xgb_pred))
    }
    
    # 최고 모델 선택
    best_model_name = max(models.keys(), key=lambda x: models[x]['r2'])
    best_model_info = models[best_model_name]
    
    print(f"   ✅ 최고 성능: {best_model_name.upper()} (R²={best_model_info['r2']:.3f}, MAE={best_model_info['mae']:.2f})")
    
    return {
        'biomarker': target_biomarker,
        'model': best_model_info['model'],
        'scaler': scaler,
        'features': feature_cols,
        'r2': best_model_info['r2'],
        'mae': best_model_info['mae'],
        'rmse': best_model_info['rmse'],
        'n_samples': len(X),
        'model_type': best_model_name.upper()
    }

# 모든 바이오마커에 대해 훈련
print("🚀 변화량 예측 모델 훈련 시작...")
print("="*80)

change_models = {}
for biomarker in biomarker_vars:
    result = train_change_model(change_df, biomarker, diet_vars)
    if result:
        change_models[biomarker] = result

print(f"\n🎉 훈련 완료: {len(change_models)}개 모델")

## 5. 모델 성능 평가 및 비교

In [ ]:
# 성능 요약
if change_models:
    performance_df = pd.DataFrame([
        {
            'Biomarker': info['biomarker'],
            'R²': info['r2'],
            'MAE': info['mae'],
            'RMSE': info['rmse'],
            'N': info['n_samples'],
            'Model': info['model_type']
        }
        for info in change_models.values()
    ])
    
    print("📊 변화량 예측 모델 성능:")
    print("="*80)
    display(performance_df)
    
    # 시각화
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # R² 비교
    colors = ['green' if r2 >= 0.1 else 'orange' if r2 >= 0.05 else 'red' 
              for r2 in performance_df['R²']]
    ax1.barh(performance_df['Biomarker'], performance_df['R²'], color=colors, alpha=0.7)
    ax1.set_xlabel('R² Score')
    ax1.set_title('변화량 예측 모델 성능 (R²)')
    ax1.axvline(0.1, color='green', linestyle='--', alpha=0.5, label='R²=0.1')
    ax1.axvline(0.05, color='orange', linestyle='--', alpha=0.5, label='R²=0.05')
    ax1.legend()
    
    # MAE 비교
    ax2.barh(performance_df['Biomarker'], performance_df['MAE'], color='skyblue', alpha=0.7)
    ax2.set_xlabel('Mean Absolute Error')
    ax2.set_title('평균 절대 오차 (MAE)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 평가:")
    good_models = performance_df[performance_df['R²'] >= 0.1]
    fair_models = performance_df[(performance_df['R²'] >= 0.05) & (performance_df['R²'] < 0.1)]
    poor_models = performance_df[performance_df['R²'] < 0.05]
    
    print(f"   🟢 양호 (R² ≥ 0.1): {len(good_models)}개")
    if len(good_models) > 0:
        print(f"      {', '.join(good_models['Biomarker'])}")
    
    print(f"   🟡 보통 (0.05 ≤ R² < 0.1): {len(fair_models)}개")
    if len(fair_models) > 0:
        print(f"      {', '.join(fair_models['Biomarker'])}")
    
    print(f"   🔴 낮음 (R² < 0.05): {len(poor_models)}개")
    if len(poor_models) > 0:
        print(f"      {', '.join(poor_models['Biomarker'])}")
else:
    print("❌ 훈련된 모델이 없습니다.")

## 6. 특성 중요도 분석

In [ ]:
# 주요 바이오마커 특성 중요도 분석
target_biomarkers_for_analysis = ['SBP', 'DBP', 'GLUCOSE', 'HBA1C']

for biomarker in target_biomarkers_for_analysis:
    if biomarker not in change_models:
        continue
    
    model_info = change_models[biomarker]
    model = model_info['model']
    features = model_info['features']
    
    print(f"\n🔍 {biomarker} 특성 중요도 (R²={model_info['r2']:.3f})")
    print("="*80)
    
    # Feature importance 추출
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'Feature': features,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        # 상위 15개 특성
        top_features = feature_importance_df.head(15)
        
        # 시각화
        plt.figure(figsize=(12, 6))
        colors = ['red' if 'Δ' in feat else 'blue' for feat in top_features['Feature']]
        plt.barh(top_features['Feature'], top_features['Importance'], color=colors, alpha=0.7)
        plt.xlabel('Importance')
        plt.title(f'{biomarker} 변화 예측 - 상위 특성 (빨강: 변화량, 파랑: baseline)')
        plt.tight_layout()
        plt.show()
        
        # 식습관 변화 특성 중요도
        diet_change_features = feature_importance_df[
            feature_importance_df['Feature'].str.startswith('Δ') &
            feature_importance_df['Feature'].str.contains('|'.join(diet_vars))
        ]
        
        if len(diet_change_features) > 0:
            print(f"\n🍽️ 주요 식습관 변화 특성 (상위 5개):")
            for i, (_, row) in enumerate(diet_change_features.head(5).iterrows(), 1):
                print(f"   {i}. {row['Feature']}: {row['Importance']:.4f}")
        else:
            print(f"\n⚠️  식습관 변화 특성의 중요도가 낮습니다.")

## 7. 모델 저장

In [ ]:
import pickle
import os

if change_models:
    # 저장할 데이터 준비
    save_data = {
        'change_models': change_models,
        'diet_vars': diet_vars,
        'biomarker_vars': biomarker_vars,
        'model_type': 'change_prediction',
        'description': '식습관 변화 → 바이오마커 변화 예측 모델'
    }
    
    # 저장
    models_dir = '../models'
    os.makedirs(models_dir, exist_ok=True)
    
    model_path = os.path.join(models_dir, 'diet_change_models.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(save_data, f)
    
    print("✅ 변화량 예측 모델 저장 완료!")
    print(f"   저장 경로: {model_path}")
    print(f"   저장된 모델 수: {len(change_models)}개")
    print(f"   파일 크기: {os.path.getsize(model_path) / (1024*1024):.2f} MB")
    
    print(f"\n📋 저장된 바이오마커:")
    for biomarker, info in change_models.items():
        print(f"   - {biomarker}: R²={info['r2']:.3f}, MAE={info['mae']:.2f}")
else:
    print("❌ 저장할 모델이 없습니다.")

## 8. 결론 및 다음 단계

### 이 모델의 장점:
1. ✅ **식습관 변화에 직접 반응**: Δ식습관 → Δ바이오마커 직접 모델링
2. ✅ **인구통계 영향 제어**: 고정 변수로 사용하여 순수 식습관 효과 추출
3. ✅ **실제 변화 데이터 사용**: 종단 데이터의 실제 변화 패턴 학습
4. ✅ **추천 시스템에 적용 가능**: 식습관 변화 제안 시 예상 효과 예측

### 한계:
1. ⚠️ R² 성능이 낮을 수 있음 (변화량 예측은 어려움)
2. ⚠️ 데이터 수가 제한적 (2회 이상 방문자만)
3. ⚠️ 시간 간격에 따른 영향 고려 필요

### 다음 단계:
1. 이 모델을 추천 시스템에 통합
2. 기존 절대값 예측 모델과 앙상블
3. 성능이 낮은 바이오마커는 규칙 기반 추천 추가